### Creating class for strategy

In [59]:
import pandas as pd
import numpy as np
import tpqoa
import time as timeout
from datetime import datetime, timedelta
import PrepareData_v4 as PD
api = tpqoa.tpqoa("oanda.cfg")

class BreakOutTrade(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, timeframe, bar_length, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.timeframe = timeframe
        self.bar_length = pd.to_timedelta(bar_length)
        self.units = units
        self.open_bar = 0 # open bar price, required to open position
        self.buy_price = 0 # to remember last ask price
        self.sell_price = 0 # to remember last bid price
        self.sl_price = 0 # stop loss price, required to validate if trade is still open
        self.tp_price = 0 # take profit price, required to validate if trade is still open
    
    def prepare_data(self):
        now = datetime.utcnow()
        now = now - timedelta(microseconds = now.microsecond)
        date_now = "{}-{}-{} {}:{}:{}".format(now.year, now.month, now.day, now.hour, now.minute, now.second)
        past = now - timedelta(days = 5) # two days of data is enough
        date_past = "{}-{}-{} {}:{}:{}".format(past.year, past.month, past.day, past.hour, past.minute, past.second)
        
        # downloading data
        try:
            data = api.get_history(instrument = instrument, start = date_past, end = date_now,
                                granularity = 'M3', price = "M", localize = False)
            data = data.drop('complete', axis=1)
        # resample data to required timeframe
        #data = data.resample("3min", label="right").last().dropna().iloc[:-1]
        except:
            print('Error while downloading data, lets try again')
            timeout.sleep(2)
            pass
        
        self.data = PD.prepare_data(data) # use imported module
        self.last_bar = self.data.index[-1] # change the time details of last bar
        return self.data
    
    # on_success function runs everytime there is a tick, so we need to make sure it is efficient
    def on_success(self, time, bid, ask):
        '''Method called on each tick of price. 
        '''
        global position
        if self.ticks % 2 ==0:
            print(self.ticks, end = " ")
            print("Time: {} | Bid: {} | Ask: {}".format(time, bid, ask))

            # prepare data & define strategy
            recent_tick = pd.to_datetime(time)
            if recent_tick - self.last_bar > self.bar_length:
                self.open_bar = (bid+ask)/2 # remember mid price of the opening bar, required to open position
                self.prepare_data()
                print("New candle added to dataset")
                
            # BUY only on the break of the last high
            if bid > self.data.highs.iloc[-1] and self.open_bar < self.data.highs.iloc[-1]:
                self.buy_price = ask
                self.buy_order()
            # SELL only on the break of the last low
            elif ask < self.data.lows.iloc[-1] and self.open_bar > self.data.lows.iloc[-1]:
                self.sell_price = bid
                self.sell_order()
                
            # Check if existing position were closed on Oanda platform
            if position == 1:
                if ask < self.sl_price:
                    position = 0
                elif bid > self.tp_price:
                    position = 0
            elif position == -1:
                if bid > self.sl_price:
                    position = 0
                elif ask < self.tp_price:
                    position = 0

    def buy_order(self):
        global position # read global variable - position
        # If both indicators are equal 1 then BUY the asset
        try:
            if self.data.dls2.iloc[-1] == 1 and self.data.dhs2.iloc[-1] == 1:
                print("Possible BUY")
                if position == 0: # BUY if no existing position
                    self.sl_price = round(trader.data.lows.iloc[-1] - 0.1,1)
                    sl_distance = round(self.buy_price - self.sl_price,1)
                    cost_of_sl = sl_distance * 0.83 # Cost of 1 pips on FR40
                    risk = self.units * 0.01 # Want to risk 1around 1% of account per trade
                    traded_units = round(risk/cost_of_sl,0) # Nr of units that can be traded
                    self.tp_price = round(self.buy_price + 3*sl_distance,1)                
                    order = self.create_order(self.instrument, traded_units,
                        sl_distance = sl_distance,
                        tp_price = self.tp_price,
                        suppress = True, ret = True)
                    position = 1
                    print("BUY {} for {} SL {} TP {}".format(traded_units, self.sell_price, self.sl_price, self.tp_price))

        except:
            print("\nConnection error while buying the asset, let's try again")
            timeout.sleep(1)
            pass
        
    def sell_order(self):
        global position # read global variable - position
        # If both indicators are equal -1 then SELL the asset
        try:
            if self.data.dhs2.iloc[-1] == -1 and self.data.dls2.iloc[-1] == -1:
                print("Possible SELL")
                # SELL if no existing position
                if position == 0:
                    self.sl_price = round(trader.data.highs.iloc[-1] + 0.1,1)
                    sl_distance = round(self.sl_price - self.sell_price,1)
                    cost_of_sl = sl_distance * 0.83 # Cost of 1 pips on FR40
                    risk = self.units * 0.01 # Want to risk 1around 1% of account per trade
                    traded_units = round(risk/cost_of_sl,0) # Nr of units that can be traded
                    self.tp_price = round(self.sell_price - 3*sl_distance,1)                
                    order = self.create_order(self.instrument, -traded_units,
                        sl_distance = sl_distance,
                        tp_price = self.tp_price,
                        suppress = True, ret = True)
                    position = -1
                    print("SELL {} for {} SL {} TP {}".format(traded_units, self.sell_price, self.sl_price, self.tp_price))

        except:
            print("\nConnection error while Selling asset, let's try again")
            timeout.sleep(1)
            pass

In [60]:
instrument = "FR40_EUR"
timeframe = "M3"
bar_length = "3m"
position = 0

In [61]:
trader = BreakOutTrade("oanda.cfg", instrument, timeframe, bar_length, 10000)

In [62]:
trader.prepare_data()

,o,h,l,c,volume,highs,lows,dhs1,dls1,dhs2,dls2,hour
time,,,,,,,,,,,,
2021-05-12 12:30:00+00:00,6266.8,6266.8,6225.0,6244.4,1638,6270.2,6225.0,-1.0,1.0,0.0,0.0,12
2021-05-12 12:33:00+00:00,6243.8,6248.8,6235.0,6247.8,262,6270.2,6225.0,-1.0,1.0,0.0,0.0,12
2021-05-12 12:36:00+00:00,6247.2,6256.4,6246.0,6252.8,115,6270.2,6225.0,-1.0,1.0,0.0,0.0,12
2021-05-12 12:39:00+00:00,6253.4,6256.6,6248.8,6256.6,79,6270.2,6225.0,-1.0,1.0,0.0,0.0,12
2021-05-12 12:42:00+00:00,6255.8,6268.2,6253.2,6268.2,61,6270.2,6225.0,-1.0,1.0,0.0,0.0,12
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-17 07:00:00+00:00,6401.5,6402.2,6396.4,6398.6,156,6421.3,6392.1,1.0,1.0,-1.0,-1.0,7
2021-05-17 07:03:00+00:00,6399.2,6402.2,6394.0,6400.8,81,6421.3,6392.1,1.0,1.0,-1.0,-1.0,7
2021-05-17 07:06:00+00:00,6401.6,6404.2,6399.8,6403.8,59,6421.3,6392.1,1.0,1.0,-1.0,-1.0,7


In [ ]:
while True:
    try:
        trader.stream_data(instrument, stop = 10000)
    except:
        print("\nError running the function, let's try again")
        timeout.sleep(3)
        pass

2 Time: 2021-05-17T07:09:25.991870338Z | Bid: 6401.4 | Ask: 6402.3
4 Time: 2021-05-17T07:09:34.262818439Z | Bid: 6400.1 | Ask: 6401.0
6 Time: 2021-05-17T07:09:37.933190591Z | Bid: 6400.1 | Ask: 6401.0
8 Time: 2021-05-17T07:09:51.540115309Z | Bid: 6401.3 | Ask: 6402.2
10 Time: 2021-05-17T07:09:53.629871227Z | Bid: 6401.4 | Ask: 6402.3
12 Time: 2021-05-17T07:10:00.110173482Z | Bid: 6402.7 | Ask: 6403.6
14 Time: 2021-05-17T07:10:03.019005779Z | Bid: 6402.7 | Ask: 6403.6
16 Time: 2021-05-17T07:10:06.203419784Z | Bid: 6403.9 | Ask: 6404.8
18 Time: 2021-05-17T07:10:08.122351040Z | Bid: 6405.2 | Ask: 6406.1
20 Time: 2021-05-17T07:10:11.538573783Z | Bid: 6405.2 | Ask: 6406.1
22 Time: 2021-05-17T07:10:21.987396093Z | Bid: 6405.4 | Ask: 6406.3
24 Time: 2021-05-17T07:10:26.672391746Z | Bid: 6406.1 | Ask: 6407.0
26 Time: 2021-05-17T07:10:41.189380637Z | Bid: 6404.7 | Ask: 6405.6
28 Time: 2021-05-17T07:10:45.783264694Z | Bid: 6405.9 | Ask: 6406.8
30 Time: 2021-05-17T07:10:55.137305365Z | Bid: 6407.

228 Time: 2021-05-17T07:23:03.765569630Z | Bid: 6382.4 | Ask: 6383.3
230 Time: 2021-05-17T07:23:07.887736710Z | Bid: 6381.2 | Ask: 6382.1
232 Time: 2021-05-17T07:23:11.998391267Z | Bid: 6381.2 | Ask: 6382.1
234 Time: 2021-05-17T07:23:27.427780343Z | Bid: 6381.2 | Ask: 6382.1
236 Time: 2021-05-17T07:24:03.821293479Z | Bid: 6382.1 | Ask: 6383.0
New candle added to dataset
238 Time: 2021-05-17T07:24:04.908632397Z | Bid: 6382.1 | Ask: 6383.0
240 Time: 2021-05-17T07:24:22.272968884Z | Bid: 6383.3 | Ask: 6384.2
242 Time: 2021-05-17T07:24:24.115638660Z | Bid: 6382.1 | Ask: 6383.0
244 Time: 2021-05-17T07:24:38.570010406Z | Bid: 6380.8 | Ask: 6381.7
246 Time: 2021-05-17T07:24:41.138999985Z | Bid: 6380.8 | Ask: 6381.7
248 Time: 2021-05-17T07:24:45.571337170Z | Bid: 6380.8 | Ask: 6381.7
250 Time: 2021-05-17T07:24:57.364845536Z | Bid: 6379.6 | Ask: 6380.5
252 Time: 2021-05-17T07:25:25.810193347Z | Bid: 6380.8 | Ask: 6381.7
254 Time: 2021-05-17T07:25:37.577375205Z | Bid: 6380.8 | Ask: 6381.7
256 Ti

462 Time: 2021-05-17T07:40:34.300381889Z | Bid: 6372.2 | Ask: 6373.1
464 Time: 2021-05-17T07:40:50.064684374Z | Bid: 6374.1 | Ask: 6375.0
466 Time: 2021-05-17T07:41:03.086014965Z | Bid: 6374.1 | Ask: 6375.0
468 Time: 2021-05-17T07:41:13.080664341Z | Bid: 6372.8 | Ask: 6373.7
470 Time: 2021-05-17T07:41:31.354011371Z | Bid: 6371.7 | Ask: 6372.6
472 Time: 2021-05-17T07:41:38.124709209Z | Bid: 6370.4 | Ask: 6371.3
474 Time: 2021-05-17T07:41:43.508900749Z | Bid: 6369.2 | Ask: 6370.1
476 Time: 2021-05-17T07:41:51.622956545Z | Bid: 6370.6 | Ask: 6371.5
478 Time: 2021-05-17T07:42:03.203177575Z | Bid: 6370.6 | Ask: 6371.5
New candle added to dataset
480 Time: 2021-05-17T07:42:15.795584867Z | Bid: 6368.6 | Ask: 6369.5
482 Time: 2021-05-17T07:42:30.053369229Z | Bid: 6369.2 | Ask: 6370.1
484 Time: 2021-05-17T07:42:42.224651357Z | Bid: 6369.3 | Ask: 6370.2
486 Time: 2021-05-17T07:43:05.551114047Z | Bid: 6368.4 | Ask: 6369.3
488 Time: 2021-05-17T07:43:15.820369707Z | Bid: 6368.3 | Ask: 6369.2
490 Ti

694 Time: 2021-05-17T08:04:16.775350686Z | Bid: 6381.7 | Ask: 6382.6
696 Time: 2021-05-17T08:04:26.257127613Z | Bid: 6380.4 | Ask: 6381.3
698 Time: 2021-05-17T08:04:42.286732339Z | Bid: 6380.4 | Ask: 6381.3
700 Time: 2021-05-17T08:04:52.870278566Z | Bid: 6381.7 | Ask: 6382.6
702 Time: 2021-05-17T08:05:09.544607532Z | Bid: 6381.7 | Ask: 6382.6
704 Time: 2021-05-17T08:05:18.616496661Z | Bid: 6382.9 | Ask: 6383.8
706 Time: 2021-05-17T08:05:27.165005406Z | Bid: 6381.7 | Ask: 6382.6
708 Time: 2021-05-17T08:05:37.322450353Z | Bid: 6380.4 | Ask: 6381.3
710 Time: 2021-05-17T08:06:17.562401203Z | Bid: 6379.8 | Ask: 6380.7
New candle added to dataset
712 Time: 2021-05-17T08:06:29.649144556Z | Bid: 6379.8 | Ask: 6380.7
714 Time: 2021-05-17T08:06:44.165812244Z | Bid: 6377.8 | Ask: 6378.7
716 Time: 2021-05-17T08:07:00.064711878Z | Bid: 6377.8 | Ask: 6378.7
718 Time: 2021-05-17T08:07:11.005455899Z | Bid: 6376.6 | Ask: 6377.5
720 Time: 2021-05-17T08:07:18.217868289Z | Bid: 6376.7 | Ask: 6377.6
722 Ti

920 Time: 2021-05-17T08:50:31.182789748Z | Bid: 6376.2 | Ask: 6377.1
922 Time: 2021-05-17T08:50:48.535513010Z | Bid: 6376.3 | Ask: 6377.2
924 Time: 2021-05-17T08:51:50.166862947Z | Bid: 6377.6 | Ask: 6378.5
New candle added to dataset
926 Time: 2021-05-17T08:52:22.554538966Z | Bid: 6378.8 | Ask: 6379.7
928 Time: 2021-05-17T08:52:40.578572885Z | Bid: 6377.6 | Ask: 6378.5
930 Time: 2021-05-17T08:54:14.412767253Z | Bid: 6377.6 | Ask: 6378.5
New candle added to dataset
932 Time: 2021-05-17T08:55:04.422711098Z | Bid: 6377.6 | Ask: 6378.5
934 Time: 2021-05-17T08:55:28.025286742Z | Bid: 6376.3 | Ask: 6377.2
936 Time: 2021-05-17T08:55:41.958784215Z | Bid: 6376.3 | Ask: 6377.2
938 Time: 2021-05-17T08:55:52.194518101Z | Bid: 6376.2 | Ask: 6377.1
940 Time: 2021-05-17T08:56:46.844185713Z | Bid: 6376.2 | Ask: 6377.1
942 Time: 2021-05-17T08:58:06.139859284Z | Bid: 6376.2 | Ask: 6377.1
New candle added to dataset
944 Time: 2021-05-17T08:59:12.459727037Z | Bid: 6376.2 | Ask: 6377.1
946 Time: 2021-05-1

...

1206 Time: 2021-05-21T11:06:10.327661544Z | Bid: 6378.9 | Ask: 6379.8
New candle added to dataset
1208 Time: 2021-05-21T11:06:55.589191020Z | Bid: 6378.3 | Ask: 6379.2
1210 Time: 2021-05-21T11:07:28.075399104Z | Bid: 6379.6 | Ask: 6380.5
1212 Time: 2021-05-21T11:08:02.666406790Z | Bid: 6379.6 | Ask: 6380.5
1214 Time: 2021-05-21T11:08:06.687690031Z | Bid: 6380.9 | Ask: 6381.8
1216 Time: 2021-05-21T11:08:30.588593201Z | Bid: 6380.9 | Ask: 6381.8
1218 Time: 2021-05-21T11:09:12.652701693Z | Bid: 6380.9 | Ask: 6381.8
New candle added to dataset
1220 Time: 2021-05-21T11:09:36.732848306Z | Bid: 6380.9 | Ask: 6381.8
1222 Time: 2021-05-21T11:10:13.564438931Z | Bid: 6381.3 | Ask: 6382.2
1224 Time: 2021-05-21T11:11:30.944430426Z | Bid: 6380.1 | Ask: 6381.0
1226 Time: 2021-05-21T11:11:51.097701961Z | Bid: 6378.6 | Ask: 6379.5
1228 Time: 2021-05-21T11:12:01.734318250Z | Bid: 6378.6 | Ask: 6379.5
New candle added to dataset
1230 Time: 2021-05-21T11:12:24.136892948Z | Bid: 6378.6 | Ask: 6379.5
1232 T

New candle added to dataset
1422 Time: 2021-05-21T12:18:22.268306520Z | Bid: 6388.3 | Ask: 6389.2
1424 Time: 2021-05-21T12:19:46.362746747Z | Bid: 6388.3 | Ask: 6389.2
1426 Time: 2021-05-21T12:20:07.752879968Z | Bid: 6389.6 | Ask: 6390.5
1428 Time: 2021-05-21T12:20:26.633761746Z | Bid: 6388.3 | Ask: 6389.2
1430 Time: 2021-05-21T12:21:04.615439231Z | Bid: 6387.1 | Ask: 6388.0
New candle added to dataset
1432 Time: 2021-05-21T12:21:18.531520739Z | Bid: 6387.1 | Ask: 6388.0
1434 Time: 2021-05-21T12:21:50.651937585Z | Bid: 6387.1 | Ask: 6388.0
1436 Time: 2021-05-21T12:22:51.051083962Z | Bid: 6385.8 | Ask: 6386.7
1438 Time: 2021-05-21T12:23:59.681403729Z | Bid: 6385.8 | Ask: 6386.7
1440 Time: 2021-05-21T12:24:43.038739416Z | Bid: 6384.6 | Ask: 6385.5
New candle added to dataset
1442 Time: 2021-05-21T12:25:26.669637215Z | Bid: 6384.6 | Ask: 6385.5
1444 Time: 2021-05-21T12:26:33.023912288Z | Bid: 6385.8 | Ask: 6386.7
1446 Time: 2021-05-21T12:27:21.462552480Z | Bid: 6384.6 | Ask: 6385.5
New ca

1636 Time: 2021-05-21T13:33:19.772223188Z | Bid: 6387.5 | Ask: 6388.4
New candle added to dataset
1638 Time: 2021-05-21T13:33:36.523163911Z | Bid: 6386.3 | Ask: 6387.2
1640 Time: 2021-05-21T13:33:47.802334678Z | Bid: 6387.6 | Ask: 6388.5
1642 Time: 2021-05-21T13:34:11.049467816Z | Bid: 6387.8 | Ask: 6388.7
1644 Time: 2021-05-21T13:34:24.978457039Z | Bid: 6386.6 | Ask: 6387.5
1646 Time: 2021-05-21T13:34:30.469498656Z | Bid: 6387.8 | Ask: 6388.7
1648 Time: 2021-05-21T13:34:58.565242468Z | Bid: 6386.6 | Ask: 6387.5
1650 Time: 2021-05-21T13:35:15.912706445Z | Bid: 6386.6 | Ask: 6387.5
1652 Time: 2021-05-21T13:35:27.028100023Z | Bid: 6386.5 | Ask: 6387.4
1654 Time: 2021-05-21T13:35:39.102587467Z | Bid: 6387.7 | Ask: 6388.6
1656 Time: 2021-05-21T13:35:46.664897312Z | Bid: 6387.7 | Ask: 6388.6
1658 Time: 2021-05-21T13:36:13.626426228Z | Bid: 6387.7 | Ask: 6388.6
New candle added to dataset
1660 Time: 2021-05-21T13:36:25.708276439Z | Bid: 6387.7 | Ask: 6388.6
1662 Time: 2021-05-21T13:36:51.859

1860 Time: 2021-05-21T14:08:16.499058557Z | Bid: 6394.4 | Ask: 6395.3
1862 Time: 2021-05-21T14:08:21.343485791Z | Bid: 6394.4 | Ask: 6395.3
1864 Time: 2021-05-21T14:08:37.314381675Z | Bid: 6394.5 | Ask: 6395.4
1866 Time: 2021-05-21T14:09:03.384572833Z | Bid: 6395.7 | Ask: 6396.6
New candle added to dataset
1868 Time: 2021-05-21T14:09:08.496421357Z | Bid: 6395.7 | Ask: 6396.6
1870 Time: 2021-05-21T14:09:16.255875032Z | Bid: 6394.4 | Ask: 6395.3
1872 Time: 2021-05-21T14:09:34.057609283Z | Bid: 6394.4 | Ask: 6395.3
1874 Time: 2021-05-21T14:09:55.111360100Z | Bid: 6394.4 | Ask: 6395.3
1876 Time: 2021-05-21T14:10:04.495801168Z | Bid: 6393.2 | Ask: 6394.1
1878 Time: 2021-05-21T14:10:11.191721395Z | Bid: 6394.4 | Ask: 6395.3
1880 Time: 2021-05-21T14:10:28.890532684Z | Bid: 6395.6 | Ask: 6396.5
1882 Time: 2021-05-21T14:10:33.173445685Z | Bid: 6396.8 | Ask: 6397.7
1884 Time: 2021-05-21T14:10:52.848592422Z | Bid: 6395.7 | Ask: 6396.6
1886 Time: 2021-05-21T14:11:01.020653902Z | Bid: 6394.4 | Ask:

2086 Time: 2021-05-21T14:41:38.877596487Z | Bid: 6392.2 | Ask: 6393.1
2088 Time: 2021-05-21T14:42:06.162560555Z | Bid: 6392.2 | Ask: 6393.1
New candle added to dataset
2090 Time: 2021-05-21T14:43:07.570505102Z | Bid: 6392.2 | Ask: 6393.1
2092 Time: 2021-05-21T14:43:18.597874735Z | Bid: 6392.2 | Ask: 6393.1
2094 Time: 2021-05-21T14:44:53.928187122Z | Bid: 6392.2 | Ask: 6393.1
2096 Time: 2021-05-21T14:45:02.730504003Z | Bid: 6392.2 | Ask: 6393.1
New candle added to dataset
2098 Time: 2021-05-21T14:45:11.885461324Z | Bid: 6392.2 | Ask: 6393.1
2100 Time: 2021-05-21T14:45:31.932981553Z | Bid: 6392.2 | Ask: 6393.1
2102 Time: 2021-05-21T14:45:39.275205552Z | Bid: 6390.9 | Ask: 6391.8
2104 Time: 2021-05-21T14:46:08.353170182Z | Bid: 6392.3 | Ask: 6393.2
2106 Time: 2021-05-21T14:46:33.109313950Z | Bid: 6391.1 | Ask: 6392.0
2108 Time: 2021-05-21T14:46:44.789441630Z | Bid: 6392.3 | Ask: 6393.2
2110 Time: 2021-05-21T14:47:38.025480635Z | Bid: 6392.3 | Ask: 6393.2
2112 Time: 2021-05-21T14:47:42.477

## dla kazdego trejdu ustalam SL i TP na podstawie ceny

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(30,15))
plt.plot(trader.data["h"]) #upper price
plt.plot(trader.data['c'], color='black',linewidth=2, markersize=12) #close prices
plt.plot(trader.data["l"]) #lower price
plt.scatter(trader.data.index, trader.data['highs'], marker = "v", color = "red")
plt.scatter(trader.data.index, trader.data['lows'], marker = "^", color = "green")
plt.show()

In [ ]:
price = 1.17822
sl_distance = round(price - trader.data.lows.iloc[-1],6)
tp_price = price + 3*sl_distance

In [ ]:
# na podstawie positionValue wiemy co zrobić z pozycją, dodać czy zamknąć wszystko
trader.create_order(instrument = trader.instrument,
                    units = 1000,
                    price = price,
                    sl_distance = sl_distance,
                    tp_price = tp_price,
                    suppress = True,
                    ret = True)

### Streamowanie strategii przez 50 ticków i na koniec zamknięcie pozycji

In [ ]:
trader.stream_data(trader.instrument, stop = 50) # trading (50 ticks)
close_order = trader.create_order(trader.instrument, units = -trader.position * trader.units, 
                                  suppress = True, ret = True) # close Final Position
trader.report_trade(close_order, "GOING NEUTRAL") # report Final Trade